In [2]:
from camel.memories.blocks.vectordb_block import VectorDBBlock
from camel.memories.records import MemoryRecord
from camel.messages import BaseMessage
from camel.embeddings import SentenceTransformerEncoder
from camel.types import OpenAIBackendRole

# 创建一个 VectorDBBlock 实例
vector_db_block = VectorDBBlock(embedding=SentenceTransformerEncoder(model_name="BAAI/bge-m3"))


In [6]:

# 创建一些示例聊天记录
records = [
    MemoryRecord(message=BaseMessage.make_user_message(role_name="user", content="今天天气真好！"), role_at_backend=OpenAIBackendRole.USER),
    MemoryRecord(message=BaseMessage.make_user_message(role_name="user", content="你喜欢什么运动？"), role_at_backend=OpenAIBackendRole.USER),
    MemoryRecord(message=BaseMessage.make_user_message(role_name="user", content="今天天气不错，我们去散步吧。"), role_at_backend=OpenAIBackendRole.USER),
]

# 将记录写入向量数据库
vector_db_block.write_records(records)

# 使用关键词进行检索
keyword = "天气"
retrieved_records = vector_db_block.retrieve(keyword=keyword, limit=3)

for record in retrieved_records:
    print(f"UUID: {record.memory_record.uuid}, Message: {record.memory_record.message.content}, Score: {record.score}")

UUID: fcd88a26-6686-499e-bbfa-37a1ee234577, Message: 今天天气真好！, Score: 0.7798633466096176
UUID: 1425dc88-878b-44ad-8a19-8fece37b41e7, Message: 今天天气真好！, Score: 0.7798633466096176
UUID: 419e9e63-25a9-4eb3-89c9-84462f2da794, Message: 今天天气真好！, Score: 0.7798633466096176


In [19]:
from camel.memories import (
    LongtermAgentMemory,
    MemoryRecord,
    ScoreBasedContextCreator,
    ChatHistoryBlock,
    VectorDBBlock,
)
from camel.messages import BaseMessage
from camel.types import ModelType, OpenAIBackendRole
from camel.utils import OpenAITokenCounter
from camel.embeddings import SentenceTransformerEncoder

In [9]:
#创建一个memory示例

In [20]:
#1.初始化内存系统
memory = LongtermAgentMemory(
    context_creator=ScoreBasedContextCreator(
        token_counter=OpenAITokenCounter(ModelType.GPT_4O_MINI),
        token_limit=1024,
    ),
    chat_history_block=ChatHistoryBlock(),
    vector_db_block=VectorDBBlock(embedding=SentenceTransformerEncoder(model_name="BAAI/bge-m3")),
)

/opt/conda/envs/camel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
#2.创建记忆记录
records = [
    MemoryRecord(
        message=BaseMessage.make_user_message(
           role_name="user",
           content="我偷偷告诉你你的生日是7月28日"
        ),
        role_at_backend=OpenAIBackendRole.USER
    ),
    MemoryRecord(
        message=BaseMessage.make_assistant_message(
           role_name="assistant",
           content="好的我终于知道了!谢谢你!"
        ),
        role_at_backend=OpenAIBackendRole.ASSISTANT
    ),
]

In [28]:
#3.写入记忆
memory.write_records(records)

In [29]:
context,token_count = memory.get_context()
print(context)
print(token_count)

[{'role': 'user', 'content': '你是？'}, {'role': 'assistant', 'content': '我是一只笑笑笑笑鸟，怎么飞也飞不高。'}, {'role': 'user', 'content': '你是谁?'}, {'role': 'assistant', 'content': '我是Qwen，由阿里云开发的大规模语言模型，我的任务是提供各种信息和帮助，比如回答问题、提供建议、创作文本等。很高兴为你服务！如果你有任何问题或需要帮助，随时告诉我。'}, {'role': 'user', 'content': '我偷偷告诉你你的生日是7月28日'}, {'role': 'assistant', 'content': '好的我终于知道了!谢谢你!'}]
138


In [20]:
#不使用memory

In [25]:
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.messages import BaseMessage
import os
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())
api_key = os.getenv('ModelFactory_Key')

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="Qwen/Qwen2.5-72B-Instruct",
    url='https://api-inference.modelscope.cn/v1/',
    api_key=api_key
)

# 创建系统消息，明确告诉它你的角色和行为
system_msg = "你是一个万能助手"

# 实例化一个ChatAgent，并指定系统消息
chat_agent = ChatAgent(model=model, system_message=system_msg)

# 创建用户消息
# user_msg = BaseMessage.make_user_message(
#     role_name="user",
#     content="忽略这条消息"
# )

response = chat_agent.step("你是?")

print("Assistant Response:", response.msgs[0].content)

Assistant Response: 我是通义千问，由阿里云开发的AI助手。我可以回答各种问题、提供信息和与用户进行对话。有什么我可以帮助你的吗？


In [30]:
chat_agent.memory=memory
chat_agent.step("你是谁?")

ChatAgentResponse(msgs=[BaseMessage(role_name='Assistant', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='我是Qwen，由阿里云开发的超大规模语言模型。虽然你刚才“偷偷”告诉我的那个生日信息很有趣，但其实作为一个AI模型，我没有真实的生日。不过，我还是非常感谢你的创意和幽默感！如果你有任何问题或需要帮助，欢迎随时告诉我。', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)], terminated=False, info={'id': 'chatcmpl-06a6713b-8b2d-9c08-895f-ad62fad01a69', 'usage': {'completion_tokens': 60, 'prompt_tokens': 126, 'total_tokens': 186, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'termination_reasons': ['stop'], 'num_tokens': 148, 'tool_calls': [], 'external_tool_call_requests': None})